In [1]:
import os
import sys
import pandas as pd
from datetime import datetime

from google.colab import drive

In [2]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
ROOT_DIR = '/content/gdrive/MyDrive/Colab Notebooks'
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'disaster_tweets')

In [4]:
print(ROOT_DIR)
print(DATA_DIR)

/content/gdrive/MyDrive/Colab Notebooks
/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets


In [5]:
if ROOT_DIR not in sys.path:
  sys.path.append(ROOT_DIR)

In [6]:
!pip install -r "$ROOT_DIR/requirementscl.txt"

In [28]:
##############
# PARAMETERS #
##############
model_name = f"roberta-base"
num_epochs = "3"
timestamp=str(datetime.now()).replace(' ','_').replace(':','').replace('-','').split('.')[0][2:-2]

model_results_name = f"{model_name}_{num_epochs}e_{timestamp}"

args = ['run_tf_text_classification.py']
args.extend([
  "--train_file", os.path.join(DATA_DIR, "train_formatted.csv"),
  "--test_file", os.path.join(DATA_DIR, "test_formatted.csv"),
  "--dev_file", os.path.join(DATA_DIR, "valid_formatted.csv"),
  "--label_column_id", "0",
  "--model_name_or_path", model_name, 
  "--output_dir", os.path.join(DATA_DIR, f"mod_{model_name}"), 
  "--num_train_epochs", num_epochs,
  "--per_device_train_batch_size", "16",
  "--per_device_eval_batch_size", "32",
  "--do_train", 
  "--do_eval",
  "--do_predict", 
  "--logging_steps", "460", ### 460 for test 256, 444 for test 512, 476 for all records in training
  "--evaluation_strategy", "steps", 
  "--save_steps", "460", ### WAS: "476", ### 2 times per epoch (476 steps per epoch)
  "--overwrite_output_dir", 
  "--max_seq_length", "128"    
])

print(f"MODEL NAME: {model_name}")
print(f"RESULTS NAME: {model_results_name}")

MODEL NAME: roberta-base
RESULTS NAME: roberta-base_3e_210319_1433


In [29]:
#!/usr/bin/env python
# coding=utf-8
# Copyright 2020 The HuggingFace Team. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" Fine-tuning the library models for sequence classification."""
import logging
import os
from dataclasses import dataclass, field
from typing import Dict, Optional

import datasets
import numpy as np
import tensorflow as tf

from transformers import (
    AutoConfig,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizer,
    TFAutoModelForSequenceClassification,
    TFTrainer,
    TFTrainingArguments,
)
from transformers.utils import logging as hf_logging


hf_logging.set_verbosity_info()
hf_logging.enable_default_handler()
hf_logging.enable_explicit_format()


def get_tfds(
    train_file: str,
    eval_file: str,
    test_file: str,
    tokenizer: PreTrainedTokenizer,
    label_column_id: int,
    max_seq_length: Optional[int] = None,
):
    files = {}

    if train_file is not None:
        files[datasets.Split.TRAIN] = [train_file]
    if eval_file is not None:
        files[datasets.Split.VALIDATION] = [eval_file]
    if test_file is not None:
        files[datasets.Split.TEST] = [test_file]

    ds = datasets.load_dataset("csv", data_files=files)
    features_name = list(ds[list(files.keys())[0]].features.keys())
    print(f"Features Name: {features_name}")
    
    label_name = features_name.pop(label_column_id)
    label_list = list(set(ds[list(files.keys())[0]][label_name]))
    label2id = {label: i for i, label in enumerate(label_list)}
    input_names = tokenizer.model_input_names
    transformed_ds = {}

    if len(features_name) == 1:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    example[features_name[0]], truncation=True, max_length=max_seq_length, padding="max_length"
                ),
                batched=True,
            )
    elif len(features_name) == 2:
        for k in files.keys():
            transformed_ds[k] = ds[k].map(
                lambda example: tokenizer.batch_encode_plus(
                    (example[features_name[0]], example[features_name[1]]),
                    truncation=True,
                    max_length=max_seq_length,
                    padding="max_length",
                ),
                batched=True,
            )

    def gen_train():
        for ex in transformed_ds[datasets.Split.TRAIN]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_val():
        for ex in transformed_ds[datasets.Split.VALIDATION]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    def gen_test():
        for ex in transformed_ds[datasets.Split.TEST]:
            d = {k: v for k, v in ex.items() if k in input_names}
            label = label2id[ex[label_name]]
            yield (d, label)

    train_ds = (
        tf.data.Dataset.from_generator(
            gen_train,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TRAIN in transformed_ds
        else None
    )

    if train_ds is not None:
        train_ds = train_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TRAIN])))

    val_ds = (
        tf.data.Dataset.from_generator(
            gen_val,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.VALIDATION in transformed_ds
        else None
    )

    if val_ds is not None:
        val_ds = val_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.VALIDATION])))

    test_ds = (
        tf.data.Dataset.from_generator(
            gen_test,
            ({k: tf.int32 for k in input_names}, tf.int64),
            ({k: tf.TensorShape([None]) for k in input_names}, tf.TensorShape([])),
        )
        if datasets.Split.TEST in transformed_ds
        else None
    )

    if test_ds is not None:
        test_ds = test_ds.apply(tf.data.experimental.assert_cardinality(len(ds[datasets.Split.TEST])))

    return train_ds, val_ds, test_ds, label2id


logger = logging.getLogger(__name__)


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.

    Using `HfArgumentParser` we can turn this class
    into argparse arguments to be able to specify them on
    the command line.
    """

    label_column_id: int = field(metadata={"help": "Which column contains the label"})
    train_file: str = field(default=None, metadata={"help": "The path of the training file"})
    dev_file: Optional[str] = field(default=None, metadata={"help": "The path of the development file"})
    test_file: Optional[str] = field(default=None, metadata={"help": "The path of the test file"})
    max_seq_length: int = field(
        default=128,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    use_fast: bool = field(default=False, metadata={"help": "Set this flag to use fast tokenization."})
    # If you want to tweak more attributes on your tokenizer, you should do it in a distinct script,
    # or just modify its tokenizer_config.json.
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )

In [30]:
sys.argv = args

# See all possible arguments in src/transformers/training_args.py
# or by passing the --help flag to this script.
# We now keep distinct sets of args, for a cleaner separation of concerns.
parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TFTrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses()

if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

# Setup logging
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(
    "n_replicas: %s, distributed training: %s, 16-bits training: %s",
    training_args.n_replicas,
    bool(training_args.n_replicas > 1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)


[INFO|training_args.py:631] 2021-03-19 14:33:46,658 >> PyTorch: setting up devices
[INFO|training_args.py:555] 2021-03-19 14:33:46,660 >> The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
[INFO|training_args_tf.py:192] 2021-03-19 14:33:46,664 >> Tensorflow: setting up strategy
03/19/2021 14:33:46 - INFO - __main__ -   n_replicas: 1, distributed training: False, 16-bits training: False
03/19/2021 14:33:46 - INFO - __main__ -   Training/evaluation parameters TFTrainingArguments(output_dir='/content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/mod_roberta-base', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=<IntervalStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=16, per_device_eval_batch

In [31]:
# Load pretrained model and tokenizer
#
# Distributed training:
# The .from_pretrained methods guarantee that only one local process can concurrently
# download model & vocab.

tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
)


[INFO|configuration_utils.py:463] 2021-03-19 14:33:47,369 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:499] 2021-03-19 14:33:47,370 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.4.2",
  "type_vocab_size": 1,
  "use_cache"

In [32]:

train_dataset, eval_dataset, test_ds, label2id = get_tfds(
    train_file=data_args.train_file,
    eval_file=data_args.dev_file,
    test_file=data_args.test_file,
    tokenizer=tokenizer,
    label_column_id=data_args.label_column_id,
    max_seq_length=data_args.max_seq_length,
)


03/19/2021 14:33:47 - WARNING - datasets.builder -   Using custom data configuration default-6a3641a1c4d40622
03/19/2021 14:33:47 - WARNING - datasets.builder -   Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-6a3641a1c4d40622/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0)
03/19/2021 14:33:47 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-6a3641a1c4d40622/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-98df6cf059ccd6ab.arrow
03/19/2021 14:33:47 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-6a3641a1c4d40622/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0/cache-07b371ff0467f778.arrow
03/19/2021 14:33:47 - WARNING - datasets.arrow_dataset -   Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-6a3641a1c4d40622/0.0.0/2

Features Name: ['target', 'text']


In [33]:
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    num_labels=len(label2id),
    label2id=label2id,
    id2label={id: label for label, id in label2id.items()},
    finetuning_task="text-classification",
    cache_dir=model_args.cache_dir,
)


[INFO|configuration_utils.py:463] 2021-03-19 14:33:50,230 >> loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
[INFO|configuration_utils.py:499] 2021-03-19 14:33:50,231 >> Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "finetuning_task": "text-classification",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": 0,
    "1": 1
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "0": 0,
    "1": 1
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  

In [34]:
with training_args.strategy.scope():
    model = TFAutoModelForSequenceClassification.from_pretrained(
        model_args.model_name_or_path,
        from_pt=bool(".bin" in model_args.model_name_or_path),
        config=config,
        cache_dir=model_args.cache_dir,
    )



[INFO|modeling_tf_utils.py:1240] 2021-03-19 14:33:50,662 >> loading weights file https://huggingface.co/roberta-base/resolve/main/tf_model.h5 from cache at /root/.cache/huggingface/transformers/22fef2e3c5012c1a8f8d7f024e30275dd2925b076abb5131dc3d1068345b6426.d409db346b0c1408865b9785d36744ccb988186626309ae8f995f86511811602.h5
[WARNING|modeling_tf_utils.py:1298] 2021-03-19 14:34:05,139 >> All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

[WARNING|modeling_tf_utils.py:1302] 2021-03-19 14:34:05,140 >> Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
def compute_metrics(p: EvalPrediction) -> Dict:
    preds = np.argmax(p.predictions, axis=1)

    return {"acc": (preds == p.label_ids).mean()}

# Initialize our Trainer
trainer = TFTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)



[INFO|trainer_tf.py:117] 2021-03-19 14:34:05,168 >> You are instantiating a Trainer but W&B is not installed. To use wandb logging, run `pip install wandb; wandb login` see https://docs.wandb.com/huggingface.
[INFO|trainer_tf.py:125] 2021-03-19 14:34:05,170 >> To use comet_ml logging, run `pip/conda install comet_ml` see https://www.comet.ml/docs/python-sdk/huggingface/


In [36]:
print(f"train: {len(train_dataset)}")
print(f"valid: {len(eval_dataset)}")
print(f"test: {len(test_ds)}")
print(f"Model name: {model_name}")
print(f"Results name: {model_results_name}")

train: 7357
valid: 256
test: 3263
Model name: roberta-base
Results name: roberta-base_3e_210319_1433


In [37]:
# Training
if training_args.do_train:
    trainer.train()
    trainer.save_model()
    tokenizer.save_pretrained(training_args.output_dir)


[INFO|trainer_tf.py:507] 2021-03-19 14:34:05,498 >> Checkpoint file /content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/mod_roberta-base/checkpoint/ckpt-2 found and restoring from checkpoint
[INFO|trainer_tf.py:516] 2021-03-19 14:34:06,553 >>   Continuing training from checkpoint, will skip to saved global_step
[INFO|trainer_tf.py:517] 2021-03-19 14:34:06,554 >>   Continuing training from epoch 2
[INFO|trainer_tf.py:518] 2021-03-19 14:34:06,556 >>   Continuing training from global step 920
[INFO|trainer_tf.py:519] 2021-03-19 14:34:06,557 >>   Will skip the first 0 steps in the first epoch
[INFO|trainer_tf.py:528] 2021-03-19 14:34:06,561 >> ***** Running training *****
[INFO|trainer_tf.py:529] 2021-03-19 14:34:06,562 >>   Num examples = 7357
[INFO|trainer_tf.py:531] 2021-03-19 14:34:06,563 >>   Num Epochs = 3
[INFO|trainer_tf.py:532] 2021-03-19 14:34:06,564 >>   Instantaneous batch size per device = 16
[INFO|trainer_tf.py:534] 2021-03-19 14:34:06,566 >>   Total train batch size

03/19/2021 14:34:09 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 14:34:09 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


03/19/2021 14:34:17 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 14:34:17 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:306] 2021-03-19 14:37:27,846 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 14:37:27,848 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 14:37:27,849 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 14:37:27,850 >>   Batch size = 32


03/19/2021 14:37:27 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 14:37:27 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:404] 2021-03-19 14:37:31,343 >> {'eval_loss': 0.6194115281105042, 'eval_acc': 0.796875, 'epoch': 3.0, 'step': 1380}
[INFO|trainer_tf.py:404] 2021-03-19 14:37:31,352 >> {'loss': 0.2004595, 'learning_rate': 0.0, 'epoch': 3.0, 'step': 1380}
[INFO|trainer_tf.py:595] 2021-03-19 14:37:38,812 >> Saving checkpoint for step 1380 at /content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/mod_roberta-base/checkpoint/ckpt-3
[INFO|trainer_tf.py:610] 2021-03-19 14:37:38,833 >> Training took: 0:03:32.255750
[INFO|trainer_tf.py:785] 2021-03-19 14:37:38,834 >> Saving model in /content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/mod_roberta-base
[INFO|configuration_utils.py:314] 2021-03-19 14:37:38,845 >> Configuration saved in /content/gdrive/MyDrive/Colab Notebooks/data/disaster_tweets/mod_roberta-base/config.json
[INFO|modeling_tf_ut

In [38]:
# Evaluation
results = {}
if training_args.do_eval:
    logger.info("*** Evaluate ***")
    result = trainer.evaluate()
    output_eval_file = os.path.join(training_args.output_dir, f"eval_results_{model_results_name}.txt")

    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results *****")

        for key, value in result.items():
            logger.info("  %s = %s", key, value)
            writer.write("%s = %s\n" % (key, value))

        results.update(result)


03/19/2021 14:38:03 - INFO - __main__ -   *** Evaluate ***
[INFO|trainer_tf.py:306] 2021-03-19 14:38:03,964 >> ***** Running Evaluation *****
[INFO|trainer_tf.py:307] 2021-03-19 14:38:03,964 >>   Num examples in dataset = 256
[INFO|trainer_tf.py:309] 2021-03-19 14:38:03,965 >>   Num examples in used in evaluation = 256
[INFO|trainer_tf.py:310] 2021-03-19 14:38:03,969 >>   Batch size = 32


03/19/2021 14:38:04 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 14:38:04 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
[INFO|trainer_tf.py:404] 2021-03-19 14:38:07,856 >> {'eval_loss': 0.6194115281105042, 'eval_acc': 0.796875, 'epoch': 3.0, 'step': 1380}
03/19/2021 14:38:07 - INFO - __main__ -   ***** Eval results *****
03/19/2021 14:38:07 - INFO - __main__ -     eval_loss = 0.6194115281105042
03/19/2021 14:38:07 - INFO - __main__ -     eval_acc = 0.796875


In [39]:
results

{'eval_acc': 0.796875, 'eval_loss': 0.6194115281105042}

In [40]:
print(label2id)

{0: 0, 1: 1}


In [41]:
# Prediction
if training_args.do_predict:
    logger.info("*** predictions ***")
    preds = trainer.predict(test_ds)

    logger.info("*** RESUTS: ***")
    logger.info(preds)
    logger.info("*** :RESUTS ***")


03/19/2021 14:38:07 - INFO - __main__ -   *** predictions ***
[INFO|trainer_tf.py:306] 2021-03-19 14:38:07,906 >> ***** Running Prediction *****
[INFO|trainer_tf.py:307] 2021-03-19 14:38:07,907 >>   Num examples in dataset = 3263
[INFO|trainer_tf.py:310] 2021-03-19 14:38:07,924 >>   Batch size = 32


03/19/2021 14:38:32 - WARNING - tensorflow -   The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).


03/19/2021 14:38:32 - WARNING - tensorflow -   The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
03/19/2021 14:38:34 - INFO - __main__ -   *** RESUTS: ***
03/19/2021 14:38:34 - INFO - __main__ -   PredictionOutput(predictions=array([[-2.328225 ,  2.0874493],
       [-2.172442 ,  1.8221797],
       [-2.8755362,  2.4670014],
       ...,
       [-3.0634918,  2.8123376],
       [-1.0641627,  1.3007073],
       [-1.9410375,  1.8270208]], dtype=float32), label_ids=array([0, 0, 0, ..., 0, 0, 0]), metrics={'eval_loss': 1.8061743343577665, 'eval_acc': 0.5822862396567576})
03/19/2021 14:38:34 - INFO - __main__ -   *** :RESUTS ***


In [42]:
decoded_preds = np.argmax(preds.predictions, axis=1)

In [43]:
decoded_preds[10:30]

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1])

In [44]:
org_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))

In [45]:
org_test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [46]:
org_test['target'] = decoded_preds

In [47]:
org_test.head()

,id,keyword,location,text,target
0,0,NaN,NaN,Just happened a terrible car crash,1
1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",1
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",1
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,1
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,0


In [48]:
org_test[['id', 'target']].to_csv(os.path.join(DATA_DIR, f'./sub_{model_results_name}.csv'), index=False)